In [5]:
# modules
from tqdm import  tqdm
from glob import glob
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import basename as bn, join, split as sp

import librosa
import parselmouth
from parselmouth.praat import call
from scipy.io.wavfile import write

import praat_formants_python as pfp


# DATASET Path and Constants

In [6]:
# Read paths
ROOT_TIMIT_DATA_PATH = "/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT"

# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Vowel info Export CSV filename
ALL_TIMIT_VOWELS_IMP_FILENAME = "a_all-timit_vowels.csv"
ALL_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, ALL_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
SUBSET_TIMIT_VOWELS_IMP_FILENAME = "b_subset-timit_vowels_vowlimLIM.csv"
SUBSET_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, SUBSET_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME = "c_timit-vowels_formant_estimation_vowlimLIM.csv"
TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH = join(ALL_EXP_FOLDER, TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME)


# TMP Audio Export folder
TEMP_AUDIO_EXP_FOLDER = "./audio_exports"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(TEMP_AUDIO_EXP_FOLDER) #make export folder

# TIMIT SAMPLING RATE
TIMIT_AUDIO_FS = 16000

### Import SUBSET TIMIT Vowel Info dataframe

In [7]:
VOWEL_LIMIT = 500
SUBSET_TIMIT_VOWELS_DF = pd.read_csv(SUBSET_TIMIT_VOWELS_IMP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"))
# SUBSET_TIMIT_VOWELS_DF.set_index("index", inplace=True)
ALL_TIMIT_VOWEL_LIST = pd.unique(SUBSET_TIMIT_VOWELS_DF["vowel_name"])
print(SUBSET_TIMIT_VOWELS_DF["vowel_name"].value_counts())
SUBSET_TIMIT_VOWELS_DF

iy      1000
ao      1000
ey      1000
ay      1000
ae      1000
ow      1000
eh      1000
ih      1000
aa      1000
er      1000
axr      927
ux       926
ah       864
oy       787
aw       781
ix       411
uw       389
uh       175
ax       112
ax-h       5
Name: vowel_name, dtype: int64


,index,audio_filepath,wav_file,person_id,sex,start_sample,end_sample,duration_sample,start_second,end_second,duration_second,vowel_name
0,0,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX282.WAV,MMDM2,M,10360,11880,1520,0.647500,0.742500,0.095000,iy
1,1,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX404.WAV,MWJG0,M,24647,26819,2172,1.540437,1.676187,0.135750,iy
2,2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX327.WAV,MMEB0,M,10330,12825,2495,0.645625,0.801562,0.155938,iy
3,3,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX129.WAV,MDLR1,M,37633,41082,3449,2.352063,2.567625,0.215562,iy
4,4,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX208.WAV,MCTT0,M,35861,38298,2437,2.241312,2.393625,0.152312,iy
...,...,...,...,...,...,...,...,...,...,...,...,...
15372,15372,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX92.WAV,MSFV0,M,40647,43446,2799,2.540437,2.715375,0.174937,ax-h
15373,15373,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX67.WAV,MJEE0,M,55915,58171,2256,3.494687,3.635687,0.141000,ax-h
15374,15374,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX403.WAV,MTMR0,M,41023,43102,2079,2.563938,2.693875,0.129938,ax-h
15375,15375,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1448.WAV,MRJM3,M,25483,27080,1597,1.592688,1.692500,0.099812,ax-h


## FUNCTION: Measure Pitch of audio chunk | PARSEL MOUTH

In [8]:
def measure_pitch(audio_path: str) -> float:
    f0min, f0max = [75, 600]
    
    sound = parselmouth.Sound(audio_path) # read the sound
    pitch = call(sound, "To Pitch", 0, f0min, f0max) #create a praat pitch object
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    return mean_pitch

## FUNCTION: Measure formants of audio chunk | PARSEL MOUTH

In [9]:
def measure_formants_psm(audio_path: str, vowel_name: str, start_sec: float, end_sec: float):
    f0min, f0max = [75, 600]
    sound = parselmouth.Sound(audio_path) # read the sound
    # pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pitch = call(sound, "To Pitch", 0.0001, f0min, f0max)
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    
    audio_chunk, fs = librosa.load(audio_path, sr=None, offset=start_sec, duration=(end_sec - start_sec))
    tmp_audio_file = os.path.join(TEMP_AUDIO_EXP_FOLDER, f"{vowel_name}.wav")
    write(tmp_audio_file, fs, audio_chunk)
    sound_frm = parselmouth.Sound(tmp_audio_file)
    # sound_frm = sound_frm.extract_part(rom_time=start_sec, to_time=end_sec, window_shape=0, relative_width=1, preserve_times=False) # read the sound chunk
    pointProcess = call(sound_frm, "To PointProcess (periodic, cc)", f0min, f0max)
    formants = call(sound_frm, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t  = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = np.mean(f1_list)
    f2_mean = np.mean(f2_list)
    f3_mean = np.mean(f3_list)
    f4_mean = np.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = np.median(f1_list)
    f2_median = np.median(f2_list)
    f3_median = np.median(f3_list)
    f4_median = np.median(f4_list)
    
    return {
        "pitch_mean_praat_base": mean_pitch,

        "F1_mean_praat_base":f1_mean,
        "F2_mean_praat_base":f2_mean,
        "F3_mean_praat_base":f3_mean,
        "F4_mean_praat_base":f4_mean,

        "F1_median_praat_base": f1_median,
        "F2_median_praat_base": f2_median,
        "F3_median_praat_base": f3_median,
        "F4_median_praat_base": f4_median,
    }

## FUNCTION: Measure formants of audio chunk | PRAAT FORMANTS

In [10]:
def measure_formants_pfp(audio_path, start_sec, end_sec):
    formants = pfp.formants_at_interval(
        audio_path, start_sec, end_sec, maxformant=5500, winlen=0.025, preemph=50
    )

    pitch_mean = measure_pitch(audio_path)
    pitch_mean = np.round(pitch_mean, 2)
    
    formants_mean = formants.mean(axis=0)
    formants_mean = list(formants_mean)[1:]  # skip time
    formants_mean = np.round(formants_mean, 2)  # round

    formants_median = np.median(formants, axis=0)
    formants_median = list(formants_median)[1:]  # skip time
    formants_median = np.round(formants_median, 2) # round


    return {
        "pitch_mean_praat_base": pitch_mean,

        "F1_mean_praat_base": formants_mean[0],
        "F2_mean_praat_base": formants_mean[1],
        "F3_mean_praat_base": formants_mean[2],

        "F1_median_praat_base": formants_median[0],
        "F2_median_praat_base": formants_median[1],
        "F3_median_praat_base": formants_median[2],
    }


### FUNCTION TESTS

In [11]:
audf, start, end, v = SUBSET_TIMIT_VOWELS_DF.loc[np.random.randint(0, len(SUBSET_TIMIT_VOWELS_DF)), ["audio_filepath", "start_second", "end_second", "vowel_name"]]
print(audf, start, end)
print(measure_formants_pfp(audf, start, end))

/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT/TRAIN/DR4/MPRK0/SX377.WAV 2.663125 2.757125
{'pitch_mean_praat_base': 151.88, 'F1_mean_praat_base': 491.01, 'F2_mean_praat_base': 1570.8, 'F3_mean_praat_base': 2486.99, 'F1_median_praat_base': 489.81, 'F2_median_praat_base': 1559.01, 'F3_median_praat_base': 2451.31}


## FUNCTION: TIMIT Vowel PITCH, FORMANT Estimation

In [12]:
def estimate_vowel_formants(vowel_info):
    audio_file = vowel_info["audio_filepath"]
    start_sec = vowel_info["start_second"]
    end_sec = vowel_info["end_second"]
    vowel_name = vowel_info["vowel_name"]

    formant_estimates = measure_formants_pfp(audio_file, start_sec, end_sec)
    # formant_estimates = measure_formants_psm(audio_file, vowel_name, start_sec, end_sec)

    new_vowel_info = dict(vowel_info) | formant_estimates

    return new_vowel_info

vinfo = SUBSET_TIMIT_VOWELS_DF.loc[np.random.randint(0, len(SUBSET_TIMIT_VOWELS_DF))]
estimate_vowel_formants(vinfo)

{'index': 8691,
 'audio_filepath': '/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT/TRAIN/DR6/FBCH0/SX236.WAV',
 'wav_file': 'SX236.WAV',
 'person_id': 'FBCH0',
 'sex': 'F',
 'start_sample': 34741,
 'end_sample': 36344,
 'duration_sample': 1603,
 'start_second': 2.1713125,
 'end_second': 2.2715,
 'duration_second': 0.1001875,
 'vowel_name': 'ix',
 'pitch_mean_praat_base': 229.59,
 'F1_mean_praat_base': 560.38,
 'F2_mean_praat_base': 1853.64,
 'F3_mean_praat_base': 3078.74,
 'F1_median_praat_base': 565.2,
 'F2_median_praat_base': 1864.1,
 'F3_median_praat_base': 3080.68}

### Create and export Vowel Formant Estimation Result dataframe

In [13]:
VOWELS_FORMANT_DF = pd.DataFrame([estimate_vowel_formants(v_i) for i, v_i in tqdm(SUBSET_TIMIT_VOWELS_DF[0:].iterrows())])

columns = ['index',  'person_id', 'sex', 'duration_second', 'vowel_name', 
           'pitch_mean_praat_base', 
           'F1_mean_praat_base', 'F2_mean_praat_base', 'F3_mean_praat_base', 
           'F1_median_praat_base', 'F2_median_praat_base', 'F3_median_praat_base']

csv_path = TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}")
if not os.path.exists(csv_path):
    VOWELS_FORMANT_DF.to_csv(csv_path, columns=columns, index=False)

VOWELS_FORMANT_DF = VOWELS_FORMANT_DF.loc[:, columns]
VOWELS_FORMANT_DF

15377it [15:42, 16.32it/s] 


,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base
0,0,MMDM2,M,0.095000,iy,93.30,423.78,1792.19,2124.79,400.73,1837.84,2131.05
1,1,MWJG0,M,0.135750,iy,92.23,331.04,2084.30,2817.60,326.05,2067.98,2838.16
2,2,MMEB0,M,0.155938,iy,128.79,376.17,2157.42,2572.08,378.88,2172.93,2483.87
3,3,MDLR1,M,0.215562,iy,119.88,451.73,2158.56,2721.53,466.94,2184.98,2736.78
4,4,MCTT0,M,0.152312,iy,143.62,378.22,2125.59,2932.89,380.77,2119.58,2908.19
...,...,...,...,...,...,...,...,...,...,...,...,...
15372,15372,MSFV0,M,0.174937,ax-h,99.52,765.19,1876.51,3195.29,600.50,1647.24,2806.36
15373,15373,MJEE0,M,0.141000,ax-h,113.59,818.58,2064.01,3234.50,414.79,1780.08,2888.72
15374,15374,MTMR0,M,0.129938,ax-h,97.79,917.57,2128.07,3541.92,1219.34,2358.93,3806.64
15375,15375,MRJM3,M,0.099812,ax-h,112.14,488.53,2069.92,2919.50,510.21,2057.41,2742.00


### Create and export Vowel Formant Estimation Result dataframe: JSON

In [ ]:
json_fp = TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}").replace(".csv", ".json")

if not os.path.exists(json_fp):
    VOWELS_FORMANT_DF.to_json(
        json_fp, index=False, orient="table"
    )

print(json_fp)